In [1]:
import os
import re
import glob
import zipfile
import fnmatch
import datetime
from zipfile import ZipFile
import pandas as pd
import numpy as np
from pandas import DataFrame
import openpyxl
from openpyxl import load_workbook

In [2]:
def create_timestamp():
    """
    Creates a timestamp in DB format.
    """
    today = datetime.date.today()
    year = today.year
    month = today.month
    day = today.day
    
    timestamp = f"{str(year)}-{str(month)}-{str(day)}"
    
    return timestamp

In [3]:
data_dir = ".\\excel\\data\\"
archive_dir = ".\\excel\\archive\\"

if not os.path.exists(archive_dir):
    os.makedirs(archive_dir)
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

In [29]:
xl_list = glob.glob(data_dir + "*.xlsx")

for xl_file in xl_list:
    sheets = pd.ExcelFile(xl_file).sheet_names
    workbook = pd.ExcelFile(xl_file)

    if fnmatch.fnmatch(xl_file.lower(), "*reason*.xlsx") == True:
        df_reason = workbook.parse(0, skiprows=2, header=None)
        df_reason.columns = ["dept", 
                             "category", 
                             "itemDesc", 
                             "itemCode", 
                             "outOfStock", 
                             "manufacIssue",
                             "disc",
                             "other",
                             "newItemIssue"]
        df_reason["itemCode"] = df_reason["itemCode"].map('{:0>6}'.format)
        df_reason["max"] = df_reason[[df_reason.columns[4], 
                                      df_reason.columns[5], 
                                      df_reason.columns[6], 
                                      df_reason.columns[7], 
                                      df_reason.columns[8]]].max(axis=1)
        df_reason.loc[df_reason["max"] == df_reason["outOfStock"], "primaryReason"] = "Out Of Stock"
        df_reason.loc[df_reason["max"] == df_reason["manufacIssue"], "primaryReason"] = "Manufacturer Issue"
        df_reason.loc[df_reason["max"] == df_reason["disc"], "primaryReason"] = "Discontinued"
        df_reason.loc[df_reason["max"] == df_reason["other"], "primaryReason"] = "Other"
        df_reason.loc[df_reason["max"] == df_reason["newItemIssue"], "primaryReason"] = "New Item Issue"
        df_reason.sort_values(by=["max"], ascending=False, inplace=True)
        df_reason.drop(columns=["dept", 
                                "category", 
                                "itemDesc", 
                                "outOfStock", 
                                "manufacIssue", 
                                "disc", 
                                "other", 
                                "newItemIssue", 
                                "max"], inplace=True)
        
    elif fnmatch.fnmatch(xl_file.lower(), "*short*.xlsx") == True:
        df_shorts = workbook.parse(0, skiprows=1, header=None)
        df_shorts.columns = ["itemDesc", 
                             "itemCode", 
                             "yesterdayOOS"]
        df_shorts["itemCode"] = df_shorts["itemCode"].map('{:0>6}'.format)
        df_shorts.drop(columns=["itemDesc"], inplace=True)
        
    elif fnmatch.fnmatch(xl_file.lower(), "*base*.xlsx") == True:
        df_base = workbook.parse(0, skiprows=1, header=None)
        df_base.columns = ["dept", 
                           "category", 
                           "itemDesc", 
                           "itemCode", 
                           "itemSize", 
                           "pvtLblFlag", 
                           "buyerCode", 
                           "invUnitShipped", 
                           "invCaseShipped", 
                           "storeOrdProdQty", 
                           "shortedQty", 
                           "grossSvcLvl", 
                           "netSvcLvl"]
        df_base["itemCode"] = df_base["itemCode"].map('{:0>6}'.format)
        df_base["buyerCode"] = df_base["buyerCode"] * 10
        df_base["itemDesc"] = df_base["itemDesc"] + "   " + df_base["itemSize"]
        
    elif fnmatch.fnmatch(xl_file.lower(), "*export*.xlsx") == True:
        df_cs = workbook.parse(0, skiprows=3, skipfooter=20, header=None)
#         df_cs = df_cs[(df_cs[6] != "14:HATFIELD NORTH") | (df_cs[6] != "1:BRATTLEBORO")]
#         df_cs = df_cs.filter([0, 14, 15, 17, 34])
#         df_cs.columns = ["custCode", 
#                          "poDueDate", 
#                          "poApptDate", 
#                          "inStock", 
#                          "daysOOS"]
#         df_cs["itemCode"] = df_cs["custCode"].astype(str).str[9:15]
#         df_cs.drop(columns=["custCode"], inplace=True)
#         df_cs.drop_duplicates(inplace=True)

In [25]:
df_cs[(df_cs[6] != "14:HATFIELD NORTH") | (df_cs[6] != "1:BRATTLEBORO")]

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,1.010071e+14,461239.0,SCOTT BATH TISSUE WH 20PLT,1.0,2096.0000/ SF,2096.0000//SF,KC WNY DPS LDC TRNLT:47544,46:WNY: GDC + PDC,B,T,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1526.0,48.0,2020-05-10
1,1.010071e+14,95231.0,SIMPLY DN BTH TISUE 1000CT 20R,1.0,2106.9000/ SF,2106.9000//SF,TOPCO/CASCADES:41325,46:WNY: GDC + PDC,PL,T,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1193.0,55.0,2020-05-10
2,1.010071e+14,461239.0,SCOTT BATH TISSUE WH 20PLT,1.0,2096.0000/ SF,2096.0000//SF,KC WNY DPS LDC TRNLT:47544,46:WNY: GDC + PDC,B,T,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1096.0,29.0,2020-05-11
3,1.010071e+14,546117.0,BRWNY 6 XL PICK A SIZE,1.0,324.5000// SF,324.5000//SF,GA PACIFIC/MIXING:41297,46:WNY: GDC + PDC,B,T,...,0.0,0.0,0.0,0.0,651.0,0.0,28.0,651.0,4.0,2020-05-11
4,1.010071e+14,95231.0,SIMPLY DN BTH TISUE 1000CT 20R,1.0,2106.9000/ SF,2106.9000//SF,TOPCO/CASCADES:41325,46:WNY: GDC + PDC,PL,T,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,614.0,24.0,2020-05-11
5,1.010071e+14,95231.0,SIMPLY DN BTH TISUE 1000CT 20R,1.0,2106.9000/ SF,2106.9000//SF,TOPCO/CASCADES:41325,46:WNY: GDC + PDC,PL,T,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,614.0,1.0,2020-05-11
6,1.020071e+14,409503.0,80/20 GB 10LB CHUB,1.0,60.0000//L LB,60.0000//LB,SWIFT & COMPANY/MONFORT:42284,46:WNY: GDC + PDC,B,T,...,0.0,0.0,0.0,0.0,668.0,56.0,0.0,612.0,53.0,2020-05-10
7,1.020071e+14,409503.0,80/20 GB 10LB CHUB,1.0,60.0000//L LB,60.0000//LB,SWIFT & COMPANY/MONFORT:42284,46:WNY: GDC + PDC,B,T,...,0.0,0.0,0.0,0.0,592.0,68.0,0.0,552.0,62.0,2020-05-11
8,1.020071e+14,409503.0,80/20 GB 10LB CHUB,1.0,60.0000//L LB,60.0000//LB,SWIFT & COMPANY/MONFORT:42284,46:WNY: GDC + PDC,B,T,...,0.0,0.0,0.0,0.0,30.0,2.0,0.0,552.0,2.0,2020-05-11
9,1.010071e+14,252515.0,BRWNY PT TAS SHTS 6 RL,1.0,242.0000// SF,242.0000//SF,GA PACIFIC/MIXING:41297,46:WNY: GDC + PDC,B,P,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,494.0,59.0,2020-05-10


In [28]:
df_join_1 = df_base.merge(df_reason, how="left", on="itemCode")
df_join_2 = df_join_1.merge(df_shorts, how="left", on="itemCode")
df_join_3 = df_join_2.merge(df_cs, how="left", on="itemCode")

# df_join_2["poDueDate"] = "NO CS DATA"
# df_join_2["poApptDate"] = "NO CS DATA"
# df_join_2["inStock"] = "NO CS DATA"
# df_join_2["daysOOS"] = "NO CS DATA"

df_join_3

,dept,category,itemDesc,itemCode,itemSize,pvtLblFlag,buyerCode,invUnitShipped,invCaseShipped,storeOrdProdQty,shortedQty,grossSvcLvl,netSvcLvl,primaryReason,yesterdayOOS,poDueDate,poApptDate,inStock,daysOOS
0,HOUSEHOLD,BATH TISSUE,SCOTT 1000 BATH TISS 20PK 2096 SF,279289,2096 SF,N,210,0,0,1718,1718,0.000000,0.000000,Other,1718,2020-05-07,2020-05-12,N,5.0
1,HOUSEHOLD,BATH TISSUE,SCOTT 1000 BATH TISS 20PK 2096 SF,279289,2096 SF,N,210,0,0,1718,1718,0.000000,0.000000,Other,1718,NaT,NaT,N,41.0
2,HOUSEHOLD,BATH TISSUE,SMPLY DN BATH TIS 1000CT 20RL 2106.94 SF,270138,2106.94 SF,Y,210,-24,-24,1190,1214,-0.020168,-0.020583,Other,1214,2020-05-15,NaT,Y,0.0
3,BEAUTY CARE,SOAP,TOP CARE HAND SANTZR INST 8 OZ 8 FZ,030207,8 FZ,Y,200,0,0,890,890,0.000000,0.000000,Other,914,NaT,NaT,N,23.0
4,BEAUTY CARE,SOAP,TOP CARE HAND SANITIZR W/ ALOE 8 FZ,078893,8 FZ,Y,200,0,0,879,879,0.000000,0.000000,Other,903,NaT,NaT,NaN,NaN
5,BEAUTY CARE,SOAP,TOP CARE HND SANTIZR INST WRLS 2 FZ,030201,2 FZ,Y,200,0,0,792,792,0.000000,0.000000,Other,792,NaT,NaT,N,23.0
6,MAIN MEAL,SOUP,MARUCHAN RAMEN CHICKEN 12 PK 36 OZ,238612,36 OZ,N,110,0,0,615,615,0.000000,NaN,Manufacturer Issue,617,2020-04-17,2020-05-15,N,10.0
7,MEAT FRESH,BEEF,GROUND BEEF 80% LEAN 60 LB,401020,60 LB,Y,500,41,41,582,541,0.070447,0.132258,Other,541,2020-05-13,2020-05-13,Y,0.0
8,HOUSEHOLD,PAPER TOWELS,SMPLY DN PAPER TWL SS WHT 6RL 305.02 SF,270140,305.02 SF,Y,210,140,140,634,494,0.220820,0.231788,Other,494,2020-05-15,NaT,Y,0.0
9,HOUSEHOLD,PAPER TOWELS,BRAWNY TEAR-A-SQRE 6 LRG WHT 242 SF,260103,242 SF,N,210,0,0,490,490,0.000000,0.000000,Other,490,NaT,NaT,N,7.0


In [ ]:
df_join_3.to_excel(f".\\excel\\archive\\oos-data-{create_timestamp()}.xlsx")

In [ ]:
# template = r".\\excel\\template.xlsx"

# writer = pd.ExcelWriter(template, engine='openpyxl')
# df_join_2.to_excel(writer, "Data", index=False)
# writer.save()
# writer.close()

# book = load_workbook(template)
# writer = pd.ExcelWriter(template, engine='openpyxl')
# writer.book = book
# df_join_2.to_excel(writer, sheet_name = "Data", index=False)
# writer.save()
# writer.close()

# workbook1 = openpyxl.load_workbook(".\\excel\\template.xlsx")

# writer = pd.ExcelWriter('file.xlsx', engine='openpyxl') 
# writer.book = workbook1
# df_join_2.to_excel(writer, sheet_name='Data', index=False, startrow=2, startcol=1)
 
# writer.save()
# writer.close()

# with pd.ExcelWriter(".\\excel\\template.xlsx", engine='openpyxl', mode='a') as writer:  
#     df_join_2.to_excel(writer, sheet_name='Data')
#     writer.close()